## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Anchorage,US,61.22,-149.90,63.00,48,1,6.93,clear sky
1,1,Carnarvon,AU,-24.87,113.63,55.40,62,0,10.29,clear sky
2,2,Maragogi,BR,-9.01,-35.22,75.04,82,24,10.85,few clouds
3,3,Punta Arenas,CL,-53.15,-70.92,39.20,86,75,28.86,broken clouds
4,4,Bathsheba,BB,13.22,-59.52,80.60,83,75,18.34,shower rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like on your trip? "))
max_temp = float(input("What is the maximum temperature you would like on your trip? "))

What is the minimum temperature you would like on your trip? 75
What is the maximum temperature you would like on your trip? 95


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Maragogi,BR,-9.01,-35.22,75.04,82,24,10.85,few clouds
4,4,Bathsheba,BB,13.22,-59.52,80.60,83,75,18.34,shower rain
8,8,Limbang,MY,4.75,115.00,78.80,88,75,1.12,broken clouds
11,11,Bethel,US,41.37,-73.41,81.00,74,75,9.17,broken clouds
13,13,Trairi,BR,-3.28,-39.27,78.93,72,0,14.54,clear sky
14,14,Georgetown,MY,5.41,100.34,77.00,88,40,8.05,thunderstorm with rain
17,17,Padang,ID,-0.95,100.35,77.00,79,87,4.56,overcast clouds
21,21,Hasaki,JP,35.73,140.83,80.60,94,20,4.70,few clouds
23,23,Portland,US,45.52,-122.68,79.00,40,40,9.17,scattered clouds
25,25,Hamilton,US,39.18,-84.53,86.00,48,20,12.75,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                245
City                   245
Country                245
Lat                    245
Lng                    245
Max Temp               245
Humidity               245
Cloudiness             245
Wind Speed             245
Current Description    245
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Maragogi,BR,75.04,few clouds,-9.01,-35.22,
4,Bathsheba,BB,80.60,shower rain,13.22,-59.52,
8,Limbang,MY,78.80,broken clouds,4.75,115.00,
11,Bethel,US,81.00,broken clouds,41.37,-73.41,
13,Trairi,BR,78.93,clear sky,-3.28,-39.27,
...,...,...,...,...,...,...,...
654,Kuala Lumpur,MY,78.80,few clouds,3.14,101.69,
658,Namatanai,PG,81.63,scattered clouds,-3.67,152.43,
661,Guarapari,BR,75.20,few clouds,-20.67,-40.50,
662,Carmen,MX,93.20,scattered clouds,18.63,-91.83,


In [9]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Maragogi,BR,75.04,few clouds,-9.01,-35.22,Salinas do Maragogi All Inclusive Resort
4,Bathsheba,BB,80.60,shower rain,13.22,-59.52,Atlantis Hotel
8,Limbang,MY,78.80,broken clouds,4.75,115.00,Purnama Hotel
11,Bethel,US,81.00,broken clouds,41.37,-73.41,Courtyard by Marriott Danbury
13,Trairi,BR,78.93,clear sky,-3.28,-39.27,Pousada Field Lilies - Trairi
...,...,...,...,...,...,...,...
654,Kuala Lumpur,MY,78.80,few clouds,3.14,101.69,Sheraton Imperial Kuala Lumpur Hotel
658,Namatanai,PG,81.63,scattered clouds,-3.67,152.43,Namatanai Hotel
661,Guarapari,BR,75.20,few clouds,-20.67,-40.50,Hotel Porto do Sol
662,Carmen,MX,93.20,scattered clouds,18.63,-91.83,Hotel Lossandes


In [14]:
# 8a. Create the output File (CSV)

output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))